In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
verify_few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/Food-verify.expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel-ner-qa'
model_name = "gpt-3.5-turbo"
text_column = 'context'
explanation_column = 'entity_explanation'
example_selection_label_column = 'has_all'
label_symbol = "^^^^"
few_shot_num = 8
few_shot_selection = 'label_diversity_similarity'
verify_few_shot_selection = "semantic_similarity"
use_current_explanation = False
use_ground_explanation = False
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=True
temperature=0.1
verify=True
label_version='v1'
task_version='v1'
random_shuffle_examples = True
random_shuffle_examples_seed = 1

# Generation Stage

In [2]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from question_answering_new import NewQuestionAnsweringTask
from attribute_extraction_new import NewAttributeExtractionTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
from prompt_template import load_ner_ae_prompt
from dataset_new import process_labels
import sys

In [3]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.QUESTION_ANSWERING] = NewQuestionAnsweringTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.ATTRIBUTE_EXTRACTION] = NewAttributeExtractionTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
sys.modules['autolabel.dataset'].AutolabelDataset.process_labels = process_labels
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [7]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_AE_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [20]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERAE_{few_shot_selection}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERAE_{few_shot_selection}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetNERAE_{few_shot_selection}_{model_name.split('/')[1]}"

In [21]:
sym_len = len(label_symbol)
label_prefix, label_suffix, label_description, task_guideline, output_guideline = load_ner_ae_prompt(label_symbol, label_version, task_version, True)

In [22]:
test_data = pd.read_csv(data_path)

In [23]:
if use_current_explanation and not use_ground_explanation:
    explanation_column = explanation_column + "_prediction"

In [24]:
attribute_description = [{'name': k+'_answer', 'description': f'{k} entities {v}'} for k, v in label_description.items()]

In [25]:
config = {
    "task_name": task_name,
    "task_type": "attribute_extraction",
    "dataset": {
        "text_column": text_column,
        "explanation_column": explanation_column,
        "example_selection_label_column": example_selection_label_column,
        "delimiter": ",",
        "label_description": label_description 
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": task_guideline,
        "output_guidelines": output_guideline,
        "attributes": attribute_description,
        "example_selection_labels":[
            "yes",
            "no"
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "use_current_explanation": use_current_explanation,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Input: {{{text_column}}}\nOutput: Let's think step by step.\n{{{explanation_column}}}\n"+"{output_dict}",
        "chain_of_thought": True
    }
}

config = NewAutoLabelConfig(config)

In [26]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-04-05 05:27:16 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.


In [27]:
label_columns = [i['name'] for i in attribute_description]

In [28]:
if use_current_explanation:
    ds = AutolabelDataset(test_data[[text_column, explanation_column, example_selection_label_column] + label_columns], config=config)
else:
    ds = AutolabelDataset(test_data[[text_column, example_selection_label_column] + label_columns], config=config)

In [29]:
agent.plan(ds)

2024-04-05 05:27:22 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:29 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:34 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Output()

2024-04-05 05:27:37 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:37 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:37 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:37 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:38 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:38 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:38 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:38 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:38 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:27:39 httpx INFO: HTTP Request: 

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $0.129  │
│ Number of Examples       │ 10      │
│ Average cost per example │ $0.0129 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at extracting entities that are related to foodborne illness incident from text. In the given 
text, your task is to label multiple types of entity that are related to foodborne illness incident.Note that you 
should surround the extracted entities in the text with "^^" and "^^". Use the following examples as a guide for 
your predictions and format your responses similarly.

Your answer will consist of an explanation, followed by the correct labeled sentences for all types of entity in 
json format. Note that if the given text does not include any certain type entity related to foodborne illness, 
your labeled sentence for this type of entity will be the input text.The keys of the json output and the 
definitions for all type of relevant entity are provided below: 
{
    "Food_answer": "Food entities are specific food item that caused the potential foodborne illness incident.",
    "Symptom_answer": "Symptom entities are specific symptoms experienced by the affected person as a result of the
suspected foodborne illness.",
    "Location_answer": "Location entities pretrain to the location where the affected person purchased or acquired 
the food associated with the potential foodborne illness.",
    "Keyword_answer": "Keyword entities include other relevant keywords or terms associated with foodborne 
illnesses, such as \"food poisoning\"."
}

Some examples with their output answers are provided below:

Input: @USER Poor thing . Of the few times I've been sick , food poisoning was the WORST , ever . #deadly
Output: Let's think step by step.
The text does not indicate a foodborne illness incident. So, no entity in the text is related to foodborne illness 
incident.
{"Food_answer": "@USER Poor thing . Of the few times I've been sick , food poisoning was the WORST , ever . 
#deadly", "Symptom_answer": "@USER Poor thing . Of the few times I've been sick , food poisoning was the WORST , 
ever . #deadly", "Location_answer": "@USER Poor thing . Of the few times I've been sick , food poisoning was the 
WORST , ever . #deadly", "Keyword_answer": "@USER Poor thing . Of the few times I've been sick , food poisoning was
the WORST , ever . #deadly"}

Input: @USER Awww : ( I hope it clears up . I just got food poisoning I think ~ throwing up all morning : /
Output: Let's think step by step.
The text indicates a foodborne illness incident. No Food entity in the text is relevant to foodborne illness 
incident. No Location entity in the text is relevant to foodborne illness incident. The word "throwing up" in the 
text is a Symptom entity, which is a specific symptom experienced by the affected person as a result of the 
suspected foodborne illness. The word "food poisoning" in the text is a Keyword entity, which is other relevant 
keyword or term associated with foodborne illnesses, such as "food poisoning".
{"Food_answer": "@USER Awww : ( I hope it clears up . I just got food poisoning I think ~ throwing up all morning :
/", "Symptom_answer": "@USER Awww : ( I hope it clears up . I just got food poisoning I think ~ ^^throwing up^^ all
morning : /", "Location_answer": "@USER Awww : ( I hope it clears up . I just got food poisoning I think ~ throwing
up all morning : /", "Keyword_answer": "@USER Awww : ( I hope it clears up . I just got ^^food poisoning^^ I think 
~ throwing up all morning : /"}

Input: Think I might have gotten a mild case of food-poisoning today - so that's been super fun .
Output: Let's think step by step.
The text does not indicate a foodborne illness incident. So, no entity in the text is related to foodborne illness 
incident.
{"Food_answer": "Think I might have gotten a mild case of food-poisoning today - so that's been super fun .", 
"Symptom_answer": "Think I might have gotten a mild case of food-poisoning today - so that's been super fun .", 
"Location_answer": "Think I might have gotten a mild case of food-poisoning today - so that's been super fun .", 
"Keyword_answer": "Think I might have gotten 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [30]:
# now, do the actual labeling
ds = agent.run(ds)

Output()

2024-04-05 05:29:30 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-05 05:29:30 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
2024-04-05 05:29:31 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"


Error generating from LLM: Error code: 400 - {'error': {'message': 'max_tokens is too large: 5000. This model 
supports at most 4096 completion tokens, whereas you provided 5000.', 'type': 'invalid_request_error', 'param': 
'max_tokens', 'code': None}}

AttributeError: 'str' object has no attribute 'items'

In [19]:
metrics = ds.eval()

┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ Food_a… ┃ Food_… ┃ Food_a… ┃ Sympt… ┃ Sympto… ┃ Sympt… ┃ Locati… ┃ Locat… ┃ Locati… ┃ Keywo… ┃ Keywor… ┃ Keywo… ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 10      │ 1.0    │ 0.8     │ 10     │ 1.0     │ 0.9    │ 10      │ 1.0    │ 0.8     │ 10     │ 1.0     │ 0.7    │
└─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┘

In [20]:
ds.df.to_csv(output_path, index=False)
ds.df.to_pickle(output_path.replace('.csv', '.pkl'))

In [21]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []
    
if not verify:
    raise StopExecution

# Verification Stage

In [ ]:
from utils import get_predictions, check_prediction, get_verify_df, construct_results

In [ ]:
verify_question_column = f'{label_type}_verify_question'
verify_answer_column = f'{label_type}_verify_answer'
pos_column = f'{label_column}_pos'

In [ ]:
test_data[[verify_question_column, pos_column]] = ds.df[[f'{label_column}_label']].apply(lambda x: get_predictions(x[f'{label_column}_label'], label_symbol), axis=1,result_type="expand")

In [ ]:
test_data[verify_answer_column] = test_data[['CategorizedLabels', verify_question_column]].apply(lambda x: check_prediction(x['CategorizedLabels'], x[verify_question_column], label_type), axis=1)

In [ ]:
new_test_data = get_verify_df(test_data, verify_question_column, verify_answer_column, text_column, pos_column)

In [ ]:
verify_task_guideline = (f"You are an expert at identifying {label_type} entities that are related to foodborne illness incident from text. " 
                         f"In the given text, your task is to verify if a given word is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')} in the given text. "
                         "Use the following examples as a guide for your analysis and format your responses similarly.")
question = f'''Do you think the word "{{{verify_question_column}}}" in the given text is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')}?'''

In [ ]:
output_verify_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_verify_' + os.path.split(data_path)[-1]
output_verify_path = os.path.join(output_dir, output_verify_name)
output_final_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_final_' + os.path.split(data_path)[-1]
output_final_path = os.path.join(output_dir, output_final_name)

In [ ]:
config = {
    "task_name": task_name+'_verification',
    "task_type": "classification",
    "dataset": {
        "label_column": verify_answer_column,
        "text_column": text_column,
        "delimiter": ",",
        "label_description": label_description 
    },
    "model": {
        "provider": em_provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": verify_task_guideline,
        "output_guidelines": f'''You will answer with just the correct output label ("Yes" or "No") and nothing else.''',
        "labels": [
            "Yes",
            "No"
        ],
        "few_shot_examples": verify_few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Context: {{{text_column}}}\nQuestion: {question}\nAnswer: {{{verify_answer_column}}}"
    }
}

config = NewAutoLabelConfig(config)

In [ ]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

In [ ]:
verify_ds = AutolabelDataset(new_test_data.drop(['text_idx',pos_column], axis=1), config=config)

In [ ]:
agent.plan(verify_ds)

In [ ]:
# now, do the actual labeling
verify_ds = agent.run(verify_ds)

In [ ]:
metrics = verify_ds.eval()

In [ ]:
new_test_data[f'{verify_answer_column}_label'] = verify_ds.df[f'{verify_answer_column}_label']

In [ ]:
verify_ds.df['text_idx'] = new_test_data['text_idx']

In [ ]:
verify_ds.df.to_csv(output_verify_path, index=False)
verify_ds.df.to_pickle(output_verify_path.replace('.csv', '.pkl'))

In [ ]:
new_result = construct_results(ds.df, new_test_data, f'{label_column}_label', f'{verify_answer_column}_label', "text_idx", pos_column, label_symbol)

In [ ]:
metrics = ds.eval()

In [ ]:
new_result.to_csv(output_final_path, index=False)
new_result.to_pickle(output_final_path.replace('.csv', '.pkl'))